- GAN
  - 생성자(Generator)
  - 판별자(Discrimnator)
  - 둘이 서로 경쟁하면서 학습

In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
(x,y),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
x = x.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
x_train,x_val,y_train,y_val = train_test_split(x,y, stratify=y, test_size=0.2)
x_train.shape, x_val.shape, x_test.shape

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


((48000, 28, 28), (12000, 28, 28), (10000, 28, 28))

In [7]:
import matplotlib.pyplot as plt
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = images.squeeze(axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

In [8]:
# fashion mnist
import tensorflow as tf
coding_size = 30  # 생성자에 입력되는 랜덤한 크기의 벡터 , 생성자가 처음 학습할때의 시작점
Dense = tf.keras.layers.Dense  # 여러번 반복되므로...
# 생성자
generator = tf.keras.Sequential([
  Dense(100, activation='relu',kernel_initializer='he_normal'),  # he_normal 안정적인 학습
  Dense(150, activation='relu',kernel_initializer='he_normal'),
  Dense(28*28, activation='sigmoid'),
  tf.keras.layers.Reshape([28,28])
])
# 판별자
discriminator = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    Dense(150, activation='relu',kernel_initializer='he_normal'),
    Dense(100, activation='relu',kernel_initializer='he_normal'),
    Dense(1, activation='sigmoid')
])
gan = tf.keras.Sequential([generator, discriminator])  # 두 네트웍을 결합
# GAN Adam 이나 rmsprop
discriminator.compile(loss='binary_crossentropy', optimizer='rmsprop') # 진짜인지 가짜인지 구분하는 판별자
discriminator.trainable = False  # 가중치를 고정하는 역활  GAN에서 생성자만 학습되도록
gan.compile(loss='binary_crossentropy', optimizer='rmsprop')

# 배치크기  - 너무 작으면 빠르고 학습이 불안정 , 너무크면 학습이 느려질수 있지만 안정적인 학습
batch_size = 32  # 64 128
# 데이터셋
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000)  # 데이터를 하나씩 넘겨줌   1000 버퍼사이트의 크기
# 일부신경망 고정된 배치크기를 필요로 할때 유용한 방법
dataset = dataset.batch(batch_size,drop_remainder=True)  # 배치크기를 유지하기위해서 남는 데이터는 사용 안한다
dataset.prefetch(1) # 성능최적화를 위한 설정 모델 현재배치를 처리하는동안 다음배치를 미리 준비해서 처리속도 향상

<_PrefetchDataset element_spec=TensorSpec(shape=(32, 28, 28), dtype=tf.float32, name=None)>

In [9]:
# next(iter(dataset))

In [10]:
# 훈련 함수
def train_gan(gan,dataset,batch_size,codding_size, n_epochs):
  generator, discriminator = gan.layers
  for epoch in range(n_epochs):
    print(f'에포크 : {epoch+1}')
    for x_batch in dataset:
      # 판별자 훈련 - 랜덤벡터데이터 - 노이즈데이터
      noise = tf.random.normal(shape=[batch_size,codding_size])
      generated_images = generator(noise)  # 가짜이미지 생성
      # 생성된 가짜 이미지와 실제 이미지를 하나로 합침
      x_fake_and_real = tf.concat([generated_images,x_batch],axis=0)
      # 정답 레이블
      y1 = tf.constant([[0.]]*batch_size + [[1.]]*batch_size)
      # 판별자 훈련
      discriminator.train_on_batch(x_fake_and_real,y1)
      # 생성자 훈련
      noise = tf.random.normal(shape=[batch_size,codding_size])
      y2 = tf.constant([[1.]]*batch_size)  # 생성자는 판별자를 속이는 목적이므로 판별자에게 1이라고 인시되도록 생성
      gan.train_on_batch(noise,y2)  # gan 전체를 학습 - 판별자는 고정, 생성자만 학습
    # 훈련과정에서 생성자 얼마나 진짜 같은 이미지를 생성하는지 시각화
    plot_multiple_images(generated_images.numpy(), 8)
    plt.show()

In [ ]:
# 학습
train_gan(gan,dataset,batch_size, coding_size, n_epochs=10)

- GAN - CNN 추가

In [1]:
# 데이터 로드
import tensorflow as tf
from sklearn.model_selection import train_test_split
(x,y),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
x = x.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
x_train,x_val,y_train,y_val = train_test_split(x,y, stratify=y, test_size=0.2)
x_train.shape, x_val.shape, x_test.shape

((48000, 28, 28), (12000, 28, 28), (10000, 28, 28))

In [5]:
# 생성자
import tensorflow as tf
coding_size = 100
generator = tf.keras.Sequential([
    tf.keras.layers.Dense(7*7*128, input_shape=[coding_size]),
    tf.keras.layers.Reshape([7,7,128]),  # 공간을 작게 만들어서 많은 채널을 가지는 특징맵
    tf.keras.layers.BatchNormalization(),
    # 업셈플링 - 더 큰 크기의 이미지를 생성  strides=2 2배로키움  14 14 64
    tf.keras.layers.Conv2DTranspose(64,kernel_size=5,strides=2,padding='same',activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # fashion mnist channel 1 흑백 28 28 1
    # tanh 출력값을 -1 ~ 1 사이로 변경 GAN 이미지 픽셀을 -1 ~ 1 사이로  정규화 해서 사용할 예정
    tf.keras.layers.Conv2DTranspose(1,kernel_size=5,strides=2,padding='same',activation='tanh'),
])
descriminator = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,kernel_size=5,strides=2,padding='same',
                           activation=tf.keras.layers.LeakyReLU(0.2)),  # 입력이 음수이면 작은 값 0.2곱해서 출력
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(128,kernel_size=5,strides=2,padding='same',
                           activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
gan = tf.keras.Sequential([generator,descriminator])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
descriminator.compile(loss='binary_crossentropy', optimizer='adam')
descriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [8]:
# 스케일링 및 크기변경
# x_train 0 ~ 1  -1 ~ 1
x_train_scaled = x_train.reshape(-1,28,28,1)*2-1  # -1 ~ 1

In [ ]:
# 학습
import matplotlib.pyplot as plt
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices(x_train_scaled).shuffle(1000)
dataset = dataset.batch(batch_size,drop_remainder=True).prefetch(1)
train_gan(gan,dataset,batch_size,coding_size,n_epochs=10)

에포크 : 1


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


In [1]:
# 작은데이터 mnist로 실행해 보기

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
(x,y),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x = x.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
x_train,x_val,y_train,y_val = train_test_split(x,y,stratify=y,test_size=0.2)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# 생성자 모델
coding_size = 100
generator = tf.keras.Sequential([
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(28*28,activation='relu'),
    tf.keras.layers.Reshape([28,28])
])
descriminator = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
# 판별자 모델 동결 (why? 단독으로만 학습되고.. gan으로 학습할때는 동결되어서 생성자위주로 학습)
descriminator.compile(loss='binary_crossentropy',optimizer='adam')
descriminator.trainable = False
# gan 모델 정의 컴파일
gan = tf.keras.Sequential([generator,descriminator])
gan.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
import matplotlib.pyplot as plt
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = images.squeeze(axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

In [12]:
def train_gan(gan,dataset,batch_size,codding_size, n_epochs):
  generator, discriminator = gan.layers
  for epoch in range(n_epochs):
    print(f'에포크 : {epoch+1}')
    for x_batch in dataset:
      # 판별자 훈련 - 랜덤벡터데이터 - 노이즈데이터
      noise = tf.random.normal(shape=[batch_size,codding_size])
      generated_images = generator(noise)  # 가짜이미지 생성
      # 생성된 가짜 이미지와 실제 이미지를 하나로 합침
      x_fake_and_real = tf.concat([generated_images,x_batch],axis=0)
      # 정답 레이블
      y1 = tf.constant([[0.]]*batch_size + [[1.]]*batch_size)
      # 판별자 훈련
      discriminator.train_on_batch(x_fake_and_real,y1)
      # 생성자 훈련
      noise = tf.random.normal(shape=[batch_size,codding_size])
      y2 = tf.constant([[1.]]*batch_size)  # 생성자는 판별자를 속이는 목적이므로 판별자에게 1이라고 인시되도록 생성
      gan.train_on_batch(noise,y2)  # gan 전체를 학습 - 판별자는 고정, 생성자만 학습
    # 훈련과정에서 생성자 얼마나 진짜 같은 이미지를 생성하는지 시각화
    plot_multiple_images(generated_images.numpy(), 8)
    plt.show()


In [ ]:
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000)
dataset = dataset.batch(batch_size,drop_remainder=True).prefetch(1)
train_gan(gan,dataset,batch_size,coding_size,n_epochs=10)

에포크 : 1


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
